## 2. Análisis de audios como datos brutos

Este es un problema de clasificación con 4 categorias distinatas. En las primeras etapas se prepara una representación de los datos para despues implementar la maquina sobre estos datos, a priori, ciertas condiciones cardiacas tienen un mayor riasgo si no son diagnosticadas, por lo que se implementara una matriz de peso sobre la función de perdida.

https://www.kaggle.com/toregil/what-s-in-a-heartbeat de referencia

> a) Construya un dataframe con los datos a analizar. Describa el dataset y determine cuántos registros hay
por clase.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('set_a.csv')
df.shape
df.info('label')
df.groupby('label')['dataset'].count()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
dataset     176 non-null object
fname       176 non-null object
label       124 non-null object
sublabel    0 non-null float64
dtypes: float64(1), object(3)
memory usage: 5.6+ KB


label
artifact    40
extrahls    19
murmur      34
normal      31
Name: dataset, dtype: int64

In [2]:
df.head()



,dataset,fname,label,sublabel
0,a,set_a/artifact__201012172012.wav,artifact,NaN
1,a,set_a/artifact__201105040918.wav,artifact,NaN
2,a,set_a/artifact__201105041959.wav,artifact,NaN
3,a,set_a/artifact__201105051017.wav,artifact,NaN
4,a,set_a/artifact__201105060108.wav,artifact,NaN


In [3]:
df.groupby('sublabel')['dataset'].count()

Series([], Name: dataset, dtype: int64)

Contamos con una entrada de 174 datos, de los cuales 52 no tienen asignada ninguna categoría, las 4 categorias son artifact, extrahls, murmur y normal \Out[18]. No estan en la misma proppción y el parametro laber es un entero, fname es un strig con la ubicación del archivo, usaremos el codigo presentado al principio para procesar este parámetro. La columna de sublabel no es usada en este dataset.

Para leer los archivos en formato .wav se utiliza el siguiente codigo

In [4]:
from scipy.io import wavfile
def clean_filename(fname, string):
    file_name = fname.split('/')[1]
    if file_name[:2] == '__':
        file_name = string + file_name
    return file_name
SAMPLE_RATE = 44100
def load_wav_file(name, path):
    s, b = wavfile.read(path + name)
    assert s == SAMPLE_RATE
    return b

> b) Lea los archivos *.wav* y transformelos en secuencias de tiempo. Realice un *padding* de ceros al final de cada secuencia para que todas queden representadas con la misma cantidad de elementos, explique la importancia de realizar este paso.

In [14]:
def padd_zeros(array,length):
    aux = np.zeros(length)
    aux[:array.shape[0]] = array
    return aux
#for i in df.groupby('label')['dataset']:
new_df =pd.DataFrame({'file_name' : df['fname'].apply(clean_filename,string='Aunlabelledtest')});
#df['label'].fillna('unclassified')_
new_df['time_series'] = new_df['file_name'].apply(load_wav_file, path='set_a/')
new_df['len_series'] = new_df['time_series'].apply(len)
new_df['time_series']=new_df['time_series'].apply(padd_zeros,length=max(new_df['len_series']))

Al normalizar el largo de los audios, se pueden ordenar mejors los latidos sobre el espacio temporal y buscar patrones. 

> c) Manipule los datos y cambie las etiquetas de los audios por otras asignadas por un doctor experto [[4]](#refs), el cual afirma que estos cambios son requeridos. Vuelva a determinar cuántos registros hay por clase. Nótese que ahora son 3 clases ¿Explique la problemática de tener etiquetas mal asignadas en los datos? ¿Un solo dato puede afectar esto?

In [35]:
new_labels =  
labels = ['artifact','normal/extrahls', 'murmur']
new_df['target'] = [labels[i] for i in new_labels]

Si las etiquetas estan mal asignadas en los datos de entrenamiento, el programa aprendera de estos errores y generará un error de exactitud. Dependiendo del tamaño de nuestro conjuto de entrenamiento, pero de forma general un solo dato erroneo no debería afectar el resultado. 

> d) Codifique las distintas clases a valores numéricos para que puedan ser trabajados por los algoritmos
clasificadores.

In [37]:
new_df['target'] = new_df['target'].astype('category')
cat_columns = new_df.select_dtypes(['category']).columns
new_df[cat_columns] = new_df[cat_columns].apply(lambda x: x.cat.codes)

Con esto cambiamos los datos categoricos a un formato de su codigo numérico.

> e) Desordene los datos, evitando ası́ el orden en el que vienen la gran mayorı́a de las etiquetas. Cree la matriz que conforma a los datos en sus dimensiones sin preprocesar, es decir, cada ejemplo es una secuencia de amplitudes en el tiempo. ¿Las dimensiones de ésta indica que puede generar problemas? ¿De qué tipo?


In [51]:
new_df = new_df.sample(frac=1,random_state=44)
X = np.stack(new_df['time_series'].values, axis=0)
y = new_df.target.values
X.shape


(176, 396900)

Se elige solo una fracción de los datos para ser usados de entrenamiento: Nuestro nuevo conjunto de muestras es un conjunto de 176 datos del mismo largo (396900). Por la dimensión de los parametros, que es  varias veces mayor a nuestra cantidad de datos, existe un alto riesgo en caer en un overfiting. 

> f) Para pre-procesar la secuencia en el tiempo realice una transformada de fourier discreta [[5]](#refs) para pasar los datos desde el dominio de tiempos al dominio de frecuencias presentes en la señal de sonido. Visualice el cambio de representación.

In [59]:
X_fourier = np.abs(np.fft.fft(X))
X_fourier.shape

(176, 396900)

176

La transformada de fourier como bien se dijo, pasa el dominio de los datos a su dominio de frecuencias, esto nos permite estudiar un problema de menor dimension (ya que la amplitud de los sonidos es discreta y esta acotada), lo que nos ayuda a evitar el overfiting.

> g) Para seguir con el pre-procesamiento realice un muestreo representativo de los datos a través de una técnica de muestreo especializada en secuencias ¿En qué beneficia este paso? ¿Cómo podrı́a determinar si el muestro es representativo?

In [63]:
from scipy import signal
X_resampled = []
for i in range(X_fourier.shape[0]):
    sequence = X_fourier[i,:].copy()
    resampled_sequence = signal.resample(sequence, 100000)
    X_resampled.append(resampled_sequence)
X_resampled = np.array(X_resampled)
X_resampled.shape

(176, 100000)

El comando .resample(p1,p1) agrupa los parametros del conjunto p1, en este caso de los datos siguientes, agrupandolos en 100000 intervalos. finalmente agregando cada dato como una lista de paramentro al conjunto de muestras procesadas.

> h) Debido a que no hay conjunto de pruebas, y que es necesario para evaluar la calidad **final** del modelo, genérelo a través de la técnica *hold-out*

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y, test_size=0.25, random_state=42)

Separamos nuestro conjunto de datos en los que van a ser usados para el entrenamiento del programa y aquellos que van a ser usados para validación. 1/4 de los datos se van a excluir del entrenamiento para solo ser usados en la validación.

> i) Realice un proceso de estándarizar los datos para ser trabajados adecuadamente. Recuerde que solo se debe ajustar (calcular media y desviación estándar) con el conjunto de entrenamiento.

In [74]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler(with_mean=True, with_std=True)
std.fit(X_train)
X_train = std.transform(X_train)
X_test = std.transform(X_test)
print ('media=' ,np.mean(X_train),' y desviación estandar=', np.var(X_train))

media= -2.5105843330190206e-17  y desviación estandar= 0.999999999999999


Como comentario es importante que el comando .fit no se aplique sobre los dos conjuntos (test y train) de ocurrir esto, si filtra información desde el conjunto de validación al conjunto de entrenamiento. Finalmente el comando .transform toma el arreglo y transforma sus datos a un arreglo con media 0 y varianza 1

> j) Realice una reducción de dimensionalidad a través de la técnica **PCA**, para representar los datos en $d = 2$ dimensiones. Recuerde que solo se debe ajustar (encontrar las componentes principales) con el conjunto de entrenamiento. Visualice apropiadamente la proyección en 2 dimensiones.

In [76]:
from sklearn.decomposition import PCA
d=2
pca_model = PCA(n_components=d)
pca_model.fit(X_train)
X_pca_train = pca_model.transform(X_train)
X_pca_test = pca_model.transform(X_test)

> k) Entrene un modelo de Regresión Logı́stica variando el parámetro de regularización $C$ construyendo un gráfico resumen del error en función de este hiper-parámetro. Además entrene un Perceptrón, variando el hiper-parámetro de regularización $\alpha$ en el rango inverso que para la Regresión Logı́stica ¿Por qué? Contruya el mismo gráfico resumen, en función de $C$ o $\alpha$. Compare y comente lo observado.

In [79]:
Cs = [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]
alphas = [1/c for c in Cs]
from sklearn.linear_model import Perceptron,LogisticRegression
model = LogisticRegression(penalty='l2', C=c,max_iter=200)
model = Perceptron(penalty='l2', alpha=a, max_iter=200)

NameError: name 'c' is not defined

> l) Genere otra representación de los datos a través de la técnica de reducción de dimensionalidad **ICA**, con dimensionalidad $d = 2$. Recuerde que sólo se debe ajustar con el conjunto de entrenamiento, si se muestra un *warning* explique el porqué. Visualice apropiadamente la proyección en 2 dimensiones. Vuelva a realizar el item k) pero para esta nueva representación.

In [80]:
from sklearn.decomposition import FastICA
ica_model = FastICA(n_components=d)
ica_model.fit(X_train)
X_ica_train = ica_model.transform(X_train)
X_ica_test = ica_model.transform(X_test)

> m) Experimente con diferentes dimensiones $d$ para la proyección de PCA e ICA con el propósito de obtener un modelo con menor error. Construya una tabla o gráfico resumen de los errores o *accuracy*, comente.


> n) Realice otra reducción de dimensionalidad ahora a través de la técnica **LDA**, para representar los datos en $d = 2$ dimensiones. Recuerde que sólo se debe ajustar con el conjunto de entrenamiento, si se muestra un *warning* explique el porqué. Visualice apropiadamente la proyección en 2 dimensiones.

In [81]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda_model = LDA(n_components=2)
lda_model.fit(X_train,y_train)
X_lda_train = lda_model.transform(X_train)
X_lda_test = lda_model.transform(X_test)

C:\Users\rosar\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


> o) Con el propósito de encontrar el mejor modelo vuelva a realizar el item k) en el nuevo espacio generado por la representación según las $d$ dimensiones de la proyección LDA. Esta nueva representación ¿mejora o empeora el desempeño? Explique.

> p) Intente mejorar el desempeño de los algoritmos ya entrenados. Diseñe ahora sus propias cracterı́sticas (*feature crafting*) a partir de los datos brutos (secuencia de amplitudes), puede inspirarse en otros trabajos [[6]](#refs), [[7]](#refs)  si desea.